In [1]:
%%writefile EncryptSHA512.c

#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <unistd.h>

#define SALT "$6$AS$"

int main(int argc, char *argv[]){

  printf("%s\n", crypt(argv[1], SALT));

  return 0;
}

Writing EncryptSHA512.c


In [2]:
%%shell

gcc EncryptSHA512.c -lcrypt -o output

./output BM09

$6$AS$DqL1/AnbWsKs1aVr.o8VMxORzrFym5NRsjpEvLWqTcrH5xb3fZkOeBrUaH9ssN4ms0TD69idsoZdb47P8B9Xu1


In [8]:
%%writefile pass-crack.c

#define _GNU_SOURCE
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <string.h>
#include <unistd.h>
#include <crypt.h>

typedef struct {
    int threadNum;
    int startRange;
    int endRange;
    char* encryptedPwd;
} ThreadData;

volatile char* crackedPassword = NULL;
pthread_mutex_t mutex;

void* crackPassword(void* threadArg);
void getSubstring(char *dest, char *src, int start, int len);

int main(int argc, char* argv[]) {
    char* encryptedPwd = "$6$AS$DqL1/AnbWsKs1aVr.o8VMxORzrFym5NRsjpEvLWqTcrH5xb3fZkOeBrUaH9ssN4ms0TD69idsoZdb47P8B9Xu1";
    if (argc > 1) {
        encryptedPwd = argv[1];
    }

    int numThreads = 2;
    pthread_t* threads = malloc(sizeof(pthread_t) * numThreads);

    int totalCombinations = 26 * 26 * 100;
    int remainder = totalCombinations % numThreads;
    totalCombinations -= remainder;
    int combinationsPerThread = totalCombinations / numThreads;
    int currentStart = 0;

    pthread_mutex_init(&mutex, NULL);

    for (int i = 0; i < numThreads; i++) {
        ThreadData* threadData = malloc(sizeof(ThreadData));
        threadData->threadNum = i;
        threadData->encryptedPwd = encryptedPwd;
        threadData->startRange = currentStart;
        currentStart += combinationsPerThread;

        if (remainder > 0) {
            threadData->endRange = currentStart;
            currentStart++;
            remainder--;
        } else {
            threadData->endRange = currentStart - 1;
        }

        printf("Thread %d handling range %d to %d\n", i, threadData->startRange, threadData->endRange);
        pthread_create(&threads[i], NULL, crackPassword, threadData);
    }

    for (int i = 0; i < numThreads; i++) {
        pthread_join(threads[i], NULL);
    }

    if (crackedPassword != NULL) {
        printf("********\nResults:\n");
        printf("Encrypted Password: '%s'\n", encryptedPwd);
        printf("Decrypted Password: '%s'\n", crackedPassword);
    }

    free(threads);
    if (crackedPassword != NULL) {
        free((char*)crackedPassword);
    }

    pthread_mutex_destroy(&mutex);
    return 0;
}

void* crackPassword(void* threadArg) {
    ThreadData* data = (ThreadData*)threadArg;
    char salt[7];
    getSubstring(salt, data->encryptedPwd, 0, 6);

    char potentialPwd[7];
    char* encryptedAttempt;
    struct crypt_data cryptBuffer;
    cryptBuffer.initialized = 0;

    for (int i = data->startRange; i <= data->endRange; i++) {
        pthread_mutex_lock(&mutex);
        if (crackedPassword != NULL) {
            pthread_mutex_unlock(&mutex);
            pthread_exit(NULL);
        }
        pthread_mutex_unlock(&mutex);

        int secondThirdRange = 26 * 100;
        int firstCharIndex = i / secondThirdRange;
        char firstChar = (char)(firstCharIndex + 'A');
        int secondCharIndex = (i / 100) % 26;
        char secondChar = (char)(secondCharIndex + 'A');
        int number = i % 100;

        printf("Thread %d testing combination: %c%c%02d\n", data->threadNum, firstChar, secondChar, number);
        sprintf(potentialPwd, "%c%c%02d", firstChar, secondChar, number);

        encryptedAttempt = (char*)crypt_r(potentialPwd, salt, &cryptBuffer);
        if (strcmp(data->encryptedPwd, encryptedAttempt) == 0) {
            pthread_mutex_lock(&mutex);
            if (crackedPassword == NULL) {
                crackedPassword = malloc(strlen(potentialPwd) + 1);
                strcpy((char*)crackedPassword, potentialPwd);
            }
            pthread_mutex_unlock(&mutex);
            printf("Thread %d found the password: %s\n", data->threadNum, potentialPwd);
            pthread_exit(NULL);
        }
    }
    free(data);
    return NULL;
}

void getSubstring(char *dest, char *src, int start, int len) {
    memcpy(dest, src + start, len);
    *(dest + len) = '\0';
}


Writing pass-crack.c


In [9]:
%%shell

gcc pass-crack.c -lcrypt -pthread -o output

./output

Streaming output truncated to the last 5000 lines.
Thread 1 testing combination: NN04
Thread 0 testing combination: AN37
Thread 0 testing combination: AN38
Thread 1 testing combination: NN05
Thread 0 testing combination: AN39
Thread 1 testing combination: NN06
Thread 0 testing combination: AN40
Thread 1 testing combination: NN07
Thread 0 testing combination: AN41
Thread 1 testing combination: NN08
Thread 0 testing combination: AN42
Thread 1 testing combination: NN09
Thread 0 testing combination: AN43
Thread 1 testing combination: NN10
Thread 0 testing combination: AN44
Thread 1 testing combination: NN11
Thread 0 testing combination: AN45
Thread 1 testing combination: NN12
Thread 0 testing combination: AN46
Thread 1 testing combination: NN13
Thread 0 testing combination: AN47
Thread 1 testing combination: NN14
Thread 0 testing combination: AN48
Thread 1 testing combination: NN15
Thread 0 testing combination: AN49
Thread 0 testing combination: AN50
Thread 1 testing combination: NN16
Thre